In [6]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json


In [7]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)

In [8]:
def concatenate_strings(lst):
    separator = ' '
    return separator.join(lst)

In [9]:
df['targetParagraphs'] = df['targetParagraphs'].apply(concatenate_strings)
df['postText'] = df['postText'].apply(concatenate_strings)
df['spoiler'] = df['spoiler'].apply(concatenate_strings)

In [10]:
df.head(5)

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,0af11f6b-c889-4520-9372-66ba25cb7657,532quh,"Wes Welker Wanted Dinner With Tom Brady, But P...",reddit,It’ll be just like old times this weekend for ...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,","[http://pixel.wp.com/b.gif?v=noscript, http://...",http://nesn.com/2016/09/wes-welker-wanted-dinn...,"{'source': 'anonymized', 'humanSpoiler': 'They...",how about that morning we go throw?,"[[[3, 151], [3, 186]]]",[passage]
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,NASA sets date for full recovery of ozone hole,Twitter,2070 is shaping up to be a great year for Moth...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",2070,"[[[0, 0], [0, 4]]]",[phrase]
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,This is what makes employees happy -- and it's...,Twitter,"Despite common belief, money isn't the key to ...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",intellectual stimulation,"[[[1, 186], [1, 210]]]",[phrase]
3,31ecf93c-3e21-4c80-949b-aa549a046b93,844567852531286016,Passion is overrated — 7 work habits you need ...,Twitter,"It’s common wisdom. Near gospel really, and no...","‘Follow your passion’ is wrong, here are 7 hab...",There's a lot more to work that loving your job,"business, work-life, careers",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",Purpose connects us to something bigger and in...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",[multi]
4,31b108a3-c828-421a-a4b9-cf651e9ac859,814186311573766144,The perfect way to cook rice so that it's perf...,Twitter,"Boiling rice may seem simple, but there is a v...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",in a rice cooker,"[[[5, 60], [5, 76]]]",[phrase]


In [11]:
def generate_new_df(df):

    new_df = pd.DataFrame({'uuid': df['uuid'],'postText':df['postText'],'paragraph' : df['targetParagraphs'] ,'spoiler': df['spoiler']})
    return new_df
    

In [12]:

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



# sklearn

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



# Utility

import re
import string
import pickle

In [13]:
wordLemm = WordNetLemmatizer()
def processed_paragraph(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [14]:
wordLemm = WordNetLemmatizer()
def processed_postText(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    
    return ' '.join(tokens)

In [15]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
new_df = generate_new_df(df)

In [18]:
new_df[1:6]

,uuid,postText,paragraph,spoiler
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,NASA sets date for full recovery of ozone hole,2070 is shaping up to be a great year for Moth...,2070
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,This is what makes employees happy -- and it's...,"Despite common belief, money isn't the key to ...",intellectual stimulation
3,31ecf93c-3e21-4c80-949b-aa549a046b93,Passion is overrated — 7 work habits you need ...,"It’s common wisdom. Near gospel really, and no...",Purpose connects us to something bigger and in...
4,31b108a3-c828-421a-a4b9-cf651e9ac859,The perfect way to cook rice so that it's perf...,"Boiling rice may seem simple, but there is a v...",in a rice cooker
5,12e3034e-a98f-4cd2-8773-3f7974161c45,What happens if your new AirPods get lost or s...,One of the biggest surprise announcements at A...,"Apple says that if AirPods are lost or stolen,..."


In [19]:
new_df['paragraph'] = new_df['paragraph'].apply(lambda x: processed_postText(x))
new_df['postText'] = new_df['postText'].apply(lambda x: processed_postText(x))
print('Text Preprocessing complete.')

Text Preprocessing complete.


In [20]:
new_df.head(50)

,uuid,postText,paragraph,spoiler
0,0af11f6b-c889-4520-9372-66ba25cb7657,wes welker wanted dinner with tom brady but pa...,it ’ ll be just like old times this weekend fo...,how about that morning we go throw?
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,nasa sets date for full recovery of ozone hole,2070 is shaping up to be a great year for moth...,2070
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,this is what makes employees happy and its not...,despite common belief money isnt the key to em...,intellectual stimulation
3,31ecf93c-3e21-4c80-949b-aa549a046b93,passion is overrated — 7 work habits you need ...,it ’ s common wisdom near gospel really and no...,Purpose connects us to something bigger and in...
4,31b108a3-c828-421a-a4b9-cf651e9ac859,the perfect way to cook rice so that its perfe...,boiling rice may seem simple but there is a ve...,in a rice cooker
5,12e3034e-a98f-4cd2-8773-3f7974161c45,what happens if your new airpods get lost or s...,one of the biggest surprise announcements at a...,"Apple says that if AirPods are lost or stolen,..."
6,8acc9f97-846a-4c00-9483-82ddd638917d,the reason why gabor kiraly wears those tracki...,june 14th 2016 33k shares they may look like t...,"""The more good games I had in them, the more I..."
7,c4e0edb3-16f0-49eb-b3a5-cdd24c546c74,you ’ ll never believe what this family saw in...,it was cold and very foggy the temperature was...,rainbow colours in the sky and a halo spanning...
8,6db35eb1-a186-4fb2-a0b8-7b94d0396589,should you drink red wine,55 say yes cheers to your health all five of o...,Red wine is clearly the drink of choice if you...
9,e26fa6ed-e364-4666-af8e-20fbada53839,hot sauce taste test find out which we named n...,we tried 29 different kinds of hot sauce to fi...,Sriracha Hot Chili Sauce Frank's RedHot Origin...


In [21]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.5 MB/s eta 0:00:00


**Implementation of Bert Model**

In [22]:
#run this
from transformers import BertForQuestionAnswering, AutoTokenizer
from transformers import BertTokenizer

modelname = 'deepset/bert-base-cased-squad2'

model = BertForQuestionAnswering.from_pretrained(modelname)
tokenizer = BertTokenizer.from_pretrained(modelname)
#tokenizer = AutoTokenizer.from_pretrained(modelname)

In [23]:
questions= new_df['postText']
answers= new_df['spoiler']
stories = new_df['paragraph']



In [24]:
encoding = tokenizer.encode_plus(text=questions.tolist(),text_pair=stories.tolist(), max_length=512, padding="max_length", truncation="only_first", return_overflowing_tokens=True, stride=128)
inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens





We need to remove 5891 to truncate the input but the first sequence has a length 3200. Please select another truncation strategy than only_first, for instance 'longest_first' or 'only_second'.


In [25]:
len(inputs)


6403

In [29]:

from transformers import pipeline
qa_pipleline = pipeline('question-answering', model=model, tokenizer=tokenizer, device=0)

In [30]:
questions = questions.tolist()
stories = stories.tolist()

AttributeError: ignored

In [31]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=f53f5141b2187abf8716869fdbe55cdbf8d812234082c9e9c57cbf6f49d74972
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [32]:
from tqdm import tqdm
import wandb
wandb.init(sync_tensorboard=True)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [33]:

answers =[]
for i in tqdm(range(len(questions))):
  answer = qa_pipleline({
       'question': questions[i],
        'context': stories[i]
  })
  answers.append(answer)


100%|██████████| 3200/3200 [12:57<00:00,  4.11it/s]


In [34]:
ans=[]
for i in range(len(answers)):
  ans.append(answers[i]['answer'])
  print(answers[i]['answer'])


both being in the same area
2070
money
ryan holiday
youre not making sushi wash the rice
youll have to buy new ones
he constantly hungover
the aliens were invading
if you are doing light to moderate drinking
15 tapatío
24 million
jenelle evans
searching for a store name with a term like coupon or coupon code
boardwalk pipeline partners
dan simmons
photos were taken
wanted to find out
35
gold vw beetle dune
a woman who waits until her boyfriend leaves town
virginia
two of the three rituel de fille forbidden lipsticks
death threats from fellow migrants another 44 said they had been sexually assaulted
2016 the studio made the announcement via press release on tuesday
not a publicity stunt
12foot alligator
fired immediately after the incident
snapchat is taking a huge dive into video advertising
you are hoping to go into politics or already work in your local constituency
moebius strip
summer holiday
81 billion in nonus income in 2012
eagerly waiting for barbara ’ s safe delivery
visually


In [35]:

dataframe = pd.DataFrame({ 'postText': questions, 'actual_spoiler':new_df['spoiler'], 'gen_spoiler':ans })

In [36]:
dataframe

,postText,actual_spoiler,gen_spoiler
0,wes welker wanted dinner with tom brady but pa...,how about that morning we go throw?,both being in the same area
1,nasa sets date for full recovery of ozone hole,2070,2070
2,this is what makes employees happy and its not...,intellectual stimulation,money
3,passion is overrated — 7 work habits you need ...,Purpose connects us to something bigger and in...,ryan holiday
4,the perfect way to cook rice so that its perfe...,in a rice cooker,youre not making sushi wash the rice
...,...,...,...
3195,has facebooks video explosion completely shake...,it hasn’t necessarily taken the wind out of Yo...,hasn ’ t necessarily taken the wind out of you...
3196,cop is eating at a chilis when teen hands him ...,"It read, ""Thanks for keeping us safe.""",a gift card
3197,5 popular myths about visible signs of aging t...,1. Anti-wrinkle creams will erase the fine lin...,if you have a lot wrinkles in your 60s
3198,you need to see this twitter account that pred...,@beyoncefan666,beyoncefan666


**Evaluation of bert + squad2**